## Introduction à la quantization 

Laurent cetinsoy

Les réseaux de neurones prennent beaucoup de place et il peut être difficile de les faire rentrer sur certains dispositifs embarqués. 

Il existe plusieurs méthodes pour réduire la taille et augmenter la vitesse d'executer des réseaux de neurone. Par exemple il y a ce qu'on appelle la quantization et le pruning.

Dans ce notebook on va faire une introduction à la quantization avec la librairie tensorflow lite.


## Quantization post training

Dans un premier temps on va quantifier notre réseau après l'avoir entraîné normalement. 


Entraîner un réseau de neurone convolutionnel simple avec keras pour faire de la classification MNIST (ou un autre dataset simple de votre choix si (vous en avez marre de ce dataset - https://keras.io/api/datasets/)




In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

input_shape = x_train.shape[1] * x_train.shape[2]
x_train_flat = x_train.reshape(-1, input_shape)
x_test_flat = x_test.reshape(-1, input_shape)

x_train_normalized = x_train_flat / 255.0
x_test_normalized = x_test_flat / 255.0

model_relu = Sequential([
    Dense(300, activation='relu', input_shape=(input_shape,)),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])

model_relu.compile(optimizer=SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history_relu = model_relu.fit(x_train_normalized, y_train, epochs=10, validation_data=(x_test_normalized, y_test))


2023-03-26 12:26:25.286101: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 12:26:25.451530: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-26 12:26:25.451565: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-26 12:26:25.490407: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-26 12:26:27.084041: W tensorflow/stream_executor/pla

Afficher le nombre de paramètre du modèle

In [2]:
model_relu.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


Sauvegarder votre modèle et afficher la taille du fichier. Si on applique une bête règle de trois, quelle est la taille occupée par paramètre ? 

In [3]:
import os

model_relu.save('model_relu.h5')


file_size = os.path.getsize('model_relu.h5')
total_params = model_relu.count_params()
size_per_param = file_size / total_params

print(f"taille occupée par paramètre: {size_per_param} octets")

taille occupée par paramètre: 4.070815048197742 octets


On va maintenant convertir notre modèle keras en modèle tensorflow lite. 

Installer la librairie tensorflow lite créer une instance de la class TFLiteConverter à partir de votre modèle keras


In [5]:
import tensorflow as tf

model = tf.keras.models.load_model('model_relu.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

Convertir votre modèle et le sauvegarder dans un fichier nommé model.tflite. Sa taille est-elle plus petite ? 

In [6]:
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

keras_file_size = os.path.getsize('model_relu.h5')
print(f"taille  Keras: {keras_file_size} octets")

tflite_file_size = os.path.getsize('model.tflite')
print(f"taille  TensorFlow Lite: {tflite_file_size} octets")


INFO:tensorflow:Assets written to: /tmp/tmp_nlsdhve/assets
taille  Keras: 1085320 octets
taille  TensorFlow Lite: 1068532 octets
2023-03-26 12:30:11.417964: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-26 12:30:11.418009: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-26 12:30:11.418811: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp_nlsdhve
2023-03-26 12:30:11.420130: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-26 12:30:11.420164: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp_nlsdhve
2023-03-26 12:30:11.424484: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-03-26 12:30:11.425414: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-26 12:30:11.46

On va maintenant spécifier des optimisations au converter. 

1. Recréer un converter

2. modifier son attribut optimizations pour ajouter une liste d'optimisation avec la valeur tf.lite.Optimize.DEFAULT

3. Relancer la conversion du modèle, sauvegarder le modèle et regarder la taille du fichier généré

In [7]:
#Recréer un converter
optimized_converter = tf.lite.TFLiteConverter.from_keras_model(model)

#2modifier son attribut optimizations pour ajouter une liste d'optimisation avec la valeur tf.lite.Optimize.DEFAULT
optimized_converter.optimizations = [tf.lite.Optimize.DEFAULT]

#3Relancer la conversion du modèle, sauvegarder le modèle et regarder la taille du fichier généré
optimized_tflite_model = optimized_converter.convert()
with open('optimized_model.tflite', 'wb') as f:
    f.write(optimized_tflite_model)

optimized_tflite_file_size = os.path.getsize('optimized_model.tflite')
print(f"Taille du fichier modèle TensorFlow Lite optimisé: {optimized_tflite_file_size} octets")


INFO:tensorflow:Assets written to: /tmp/tmpg5ay08a4/assets
INFO:tensorflow:Assets written to: /tmp/tmpg5ay08a4/assets
Taille du fichier modèle TensorFlow Lite optimisé: 273064 octets
2023-03-26 12:32:47.217262: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-26 12:32:47.217306: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-26 12:32:47.217458: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpg5ay08a4
2023-03-26 12:32:47.218483: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-26 12:32:47.218509: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpg5ay08a4
2023-03-26 12:32:47.222556: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-26 12:32:47.249066: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on S

Quelle type  de quantization Optimize.Default, utilise-t-elle ?


## Quantization aware training 

Dans cette section on va s'intéresser à l'entraînement sensible à la quantification. L'idée est de simuler les effets de la quantification pendant l'entraînement pour que le modèle ajuste les poids afin de tenir ocmpte de la quantification. L'idée est de prendre un modèle déjà entraîné normalement et de le réentraîné en faisant un peu de quantization pendant l'entraînement. 


Reprendre le modèle entraîné sur MNIST


In [9]:
model = tf.keras.models.load_model('model_relu.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


A l'aide de la fonction quantize de tensorflow_model_optimization, créer une seconde version de votre modèle entraîné nommé qat_model

In [11]:
!pip install tensorflow_model_optimization==0.7.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.8/153.8 KB 40.5 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [12]:
import tensorflow_model_optimization as tfmot

qat_model = tfmot.quantization.keras.quantize_model(model)
qat_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 784)              3         
 er)                                                             
                                                                 
 quant_dense (QuantizeWrappe  (None, 300)              235505    
 rV2)                                                            
                                                                 
 quant_dense_1 (QuantizeWrap  (None, 100)              30105     
 perV2)                                                          
                                                                 
 quant_dense_2 (QuantizeWrap  (None, 10)               1015      
 perV2)                                                          
                                                                 
Total params: 266,628
Trainable params: 266,610
Non-trai

Compiler le modèle

In [14]:
qat_model.compile(optimizer=SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Afficher le summary du modèle. D'après vous ce modèle est-il quantifié ? 

In [15]:
qat_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 784)              3         
 er)                                                             
                                                                 
 quant_dense (QuantizeWrappe  (None, 300)              235505    
 rV2)                                                            
                                                                 
 quant_dense_1 (QuantizeWrap  (None, 100)              30105     
 perV2)                                                          
                                                                 
 quant_dense_2 (QuantizeWrap  (None, 10)               1015      
 perV2)                                                          
                                                                 
Total params: 266,628
Trainable params: 266,610
Non-trai

Réentraîner votre modèle sur un sous ensemble des données (sur une ou deux epochs) et afficher la performance sur le train et test set

In [16]:
#1 Réentraîner votre modèle sur un sous ensemble des données (sur une ou deux epochs)
history_qat = qat_model.fit(x_train_normalized[:5000], y_train[:5000], epochs=2, validation_data=(x_test_normalized[:1000], y_test[:1000]))

#2  afficher la performance sur le train et test set
train_loss, train_acc = qat_model.evaluate(x_train_normalized[:5000], y_train[:5000], verbose=2)
print(f"Performance sur le train set - Loss: {train_loss}, Accuracy: {train_acc}")

test_loss, test_acc = qat_model.evaluate(x_test_normalized[:1000], y_test[:1000], verbose=2)
print(f"Performance sur le test set - Loss: {test_loss}, Accuracy: {test_acc}")

Epoch 1/2
157/157 [==============================] - 2s 11ms/step - loss: 0.1329 - accuracy: 0.9668 - val_loss: 0.1209 - val_accuracy: 0.9710
Epoch 2/2
157/157 [==============================] - 2s 10ms/step - loss: 0.0823 - accuracy: 0.9818 - val_loss: 0.1128 - val_accuracy: 0.9710
157/157 - 0s - loss: 0.0728 - accuracy: 0.9828 - 476ms/epoch - 3ms/step
Performance sur le train set - Loss: 0.07283902913331985, Accuracy: 0.9828000068664551
32/32 - 0s - loss: 0.1128 - accuracy: 0.9710 - 109ms/epoch - 3ms/step
Performance sur le test set - Loss: 0.11279353499412537, Accuracy: 0.9710000157356262


Convertir votre modèle avec TFLite

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)
tflite_model = converter.convert()

with open('qat_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpp_fz_jwq/assets
INFO:tensorflow:Assets written to: /tmp/tmpp_fz_jwq/assets
2023-03-26 12:39:31.232708: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-26 12:39:31.232760: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-26 12:39:31.232929: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpp_fz_jwq
2023-03-26 12:39:31.235432: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-26 12:39:31.235469: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpp_fz_jwq
2023-03-26 12:39:31.251886: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-26 12:39:31.308266: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpp_fz_jwq
2023-03-26 12:39:31.32

Sauvegarder le modèle QAT et comparer les tailles des modèles

In [18]:
qat_model.save('qat_model.h5')

qat_model_file_size = os.path.getsize('qat_model.h5')
print(f"Taille du fichier modèle quantifié: {qat_model_file_size} octets")


Taille du fichier modèle quantifié: 1100392 octets


Comparer les performances des trois modèles suivants (taille et accuracy) : 
- modèle original
- modèle quantifié avec la post training quantization
- modèle entraîné avec la training aware quantization




<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3cbfad71-571a-4cfb-8f30-25f77afca130' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>